In [1]:
from tqdm import tqdm
import torch
import json
import numpy as np
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
device = torch.device('cuda:0')
torch.cuda.empty_cache()

In [4]:
%cd /DATA/ichuviliaeva/videos/i3d_experemental

/DATA/ichuviliaeva/videos/i3d_experemental


In [5]:
model_name = "i3d_r50"
model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)
model = model.to(device)
model = model.eval()

Using cache found in /home/ichuviliaeva/.cache/torch/hub/facebookresearch_pytorchvideo_master


In [6]:
import torch.nn as nn
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)

from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)

In [7]:
side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 21
sampling_rate = 1
frames_per_second = 30
alpha = 4

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size)
        ]
    ),
)

In [8]:
clip_duration = (num_frames * sampling_rate) / frames_per_second
video_path = "/DATA/ichuviliaeva/videos/50salads_vid/rgb/rgb-01-1.avi"
video = EncodedVideo.from_path(video_path)
start_sec = 0
end_sec = start_sec + clip_duration

An exception occurred in telemetry logging.Disabling telemetry to prevent further exceptions.
Traceback (most recent call last):
  File "/home/ichuviliaeva/miniconda3/envs/pytorchvideo-env/lib/python3.7/site-packages/iopath/common/file_io.py", line 946, in __log_tmetry_keys
    handler.log_event()
  File "/home/ichuviliaeva/miniconda3/envs/pytorchvideo-env/lib/python3.7/site-packages/iopath/common/event_logger.py", line 98, in log_event
    self._evt = SimpleEventRecord()
NameError: name 'SimpleEventRecord' is not defined


In [9]:
print(video._duration)

11687/30


In [10]:
print(end_sec)

0.7


In [11]:
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
video_data = transform(video_data)

In [12]:
inputs = video_data["video"]
print(len(inputs))

3


In [13]:
inputs = [i.to(device)[None, ...] for i in inputs]
print(len(inputs))

3


In [14]:
inputs = torch.cat(inputs).unsqueeze(0)

In [15]:
print(inputs.shape)

torch.Size([1, 3, 21, 256, 256])


In [16]:
res = model(inputs)
print(res.shape)

torch.Size([1, 400])


Это результат для Kinetics - надо обрезать модель

Фичи от авторов ASFormer

In [18]:
asf_features_path = "/DATA/ichuviliaeva/videos/data/50salads/features/rgb-01-1.npy"

In [19]:
asf_features = np.load(asf_features_path)
print(asf_features)
print(asf_features.shape)

[[5.7630528e-02 5.7510488e-02 5.8088370e-02 ... 1.3417178e-01
  1.4612103e-01 1.4192495e-01]
 [3.6829415e-01 3.7212294e-01 3.7371206e-01 ... 2.9188153e-01
  2.9564372e-01 2.9331002e-01]
 [1.7417416e-01 1.7289409e-01 1.7225347e-01 ... 1.2861495e-01
  1.2145682e-01 1.0264049e-01]
 ...
 [1.6405284e-03 7.7179502e-05 6.3469826e-04 ... 2.9542649e-01
  3.5986567e-01 2.1430099e-01]
 [3.9347415e-03 5.0248839e-03 8.0781560e-03 ... 4.0529925e-01
  3.3248940e-01 2.1924818e-01]
 [2.1453551e-03 1.2880345e-03 1.6674149e-03 ... 5.0064940e-02
  1.4004254e-01 6.4414769e-02]]
(2048, 11679)


Проверка на последнюю активацию.

In [20]:
np.sum(asf_features < 0)

0

да, все фичи положительные и активация была

In [17]:
model_corrected = model

In [18]:
print(model_corrected)

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=[2, 3, 3], bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=[1, 0, 0], bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kernel_size=(1,

In [19]:
# print(model_corrected.blocks[6])

In [20]:
model_block = model.blocks[6]
model_pool = model.blocks[6].pool
model_dropout = model.blocks[6].dropout
model_proj = model.blocks[6].proj
model_output_pool = model.blocks[6].output_pool

In [21]:
model_corrected.blocks[6] = model_output_pool

In [22]:
print(model_corrected)

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=[2, 3, 3], bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=[1, 0, 0], bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kernel_size=(1,

In [23]:
model_corrected = model_corrected.to(device)
model_corrected = model_corrected.eval()

In [24]:
features_corrected = model_corrected(inputs)

In [25]:
print(features_corrected.shape)

torch.Size([1, 2048, 1, 1, 1])


In [26]:
print(features_corrected.squeeze()[0:20])
print(features_corrected.squeeze().shape)

tensor([3.1444e-05, 9.4848e-02, 2.1775e-02, 4.7639e-02, 4.8868e-03, 4.4512e-02,
        1.0908e-01, 3.6642e-01, 8.8811e-02, 4.2758e-03, 1.4307e-01, 1.0361e-01,
        4.2775e-02, 3.4515e-02, 7.3974e-02, 2.3381e-02, 4.8130e-03, 2.5617e-02,
        1.7150e-01, 1.0875e-01], device='cuda:0', grad_fn=<SliceBackward>)
torch.Size([2048])


In [27]:
model_corrected.blocks[6] = nn.Sequential(
    model_pool,
    model_dropout,
    model_output_pool
)

In [28]:
print(model_corrected)

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=[2, 3, 3], bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=[1, 0, 0], bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kernel_size=(1,

In [29]:
model_corrected = model_corrected.to(device)
model_corrected = model_corrected.eval()

In [30]:
features_corrected = model_corrected(inputs)

In [31]:
print(features_corrected.shape)

torch.Size([1, 2048, 1, 1, 1])


In [32]:
print(features_corrected.squeeze()[0:20])
print(features_corrected.squeeze().shape)

tensor([7.3339e-06, 3.6088e-02, 7.8960e-03, 1.6036e-02, 2.1285e-03, 2.2434e-02,
        4.6213e-02, 2.7959e-01, 3.2626e-02, 1.5766e-03, 1.1716e-01, 4.1090e-02,
        1.6393e-02, 1.1028e-02, 3.2041e-02, 7.6254e-03, 2.0127e-03, 9.9489e-03,
        6.8912e-02, 3.3493e-02], device='cuda:0', grad_fn=<SliceBackward>)
torch.Size([2048])


In [33]:
model_corrected.blocks[6] = nn.Sequential(
    model_pool,
    model_output_pool
)

In [34]:
print(model_corrected)

Net(
  (blocks): ModuleList(
    (0): ResNetBasicStem(
      (conv): Conv3d(3, 64, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=[2, 3, 3], bias=False)
      (norm): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activation): ReLU()
      (pool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=[0, 1, 1], dilation=1, ceil_mode=False)
    )
    (1): ResStage(
      (res_blocks): ModuleList(
        (0): ResBlock(
          (branch1_conv): Conv3d(64, 256, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (branch1_norm): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (branch2): BottleneckBlock(
            (conv_a): Conv3d(64, 64, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=[1, 0, 0], bias=False)
            (norm_a): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_a): ReLU()
            (conv_b): Conv3d(64, 64, kernel_size=(1,

In [35]:
model_corrected = model_corrected.to(device)
model_corrected = model_corrected.eval()

In [36]:
features_corrected = model_corrected(inputs)

In [37]:
print(features_corrected.shape)

torch.Size([1, 2048, 1, 1, 1])


In [38]:
print(features_corrected.squeeze()[0:20])
print(features_corrected.squeeze().shape)

tensor([7.3339e-06, 3.6088e-02, 7.8960e-03, 1.6036e-02, 2.1285e-03, 2.2434e-02,
        4.6213e-02, 2.7959e-01, 3.2626e-02, 1.5766e-03, 1.1716e-01, 4.1090e-02,
        1.6393e-02, 1.1028e-02, 3.2041e-02, 7.6254e-03, 2.0127e-03, 9.9489e-03,
        6.8912e-02, 3.3493e-02], device='cuda:0', grad_fn=<SliceBackward>)
torch.Size([2048])


Странно, что оно не одинаково. Пока будет модель, где только AdaptiveAvgPool3d(output_size=1)

In [47]:
model_corrected.blocks[6] = nn.Identity()

In [48]:
features_corrected = model_corrected(inputs)
print(features_corrected.shape)

torch.Size([1, 2048, 10, 8, 8])


In [61]:
last_block_was = model_block
last_block_middle = nn.Sequential(
    model_pool,
    model_dropout,
    model_output_pool
)
last_block_want = model_output_pool

In [62]:
fake_features = torch.ones(1, 2048, 10, 8, 8).to(device)

In [63]:
res_was = last_block_was(fake_features).squeeze()
res_middle = last_block_middle(fake_features).squeeze()
res_want = last_block_want(fake_features).squeeze()
print(res_was)

tensor([-6.9382e-01,  7.9359e+00,  1.5178e-02,  2.6120e+00,  1.8328e-01,
         7.1407e+00,  3.1413e+00, -3.0289e+00, -5.5442e+00, -3.0188e-01,
        -4.6863e+00,  2.2883e+00,  2.7022e+00, -1.4828e-01,  4.5277e+00,
        -3.5822e+00,  2.8455e+00, -6.0096e+00, -3.6697e-01, -8.2406e-01,
         2.8935e+00,  3.2221e+00, -9.0040e+00, -6.2124e+00,  2.6356e-02,
        -4.3894e+00,  3.3478e+00,  3.7079e-01, -1.3285e+00, -6.3142e+00,
        -2.1650e-01,  8.1089e-01, -4.8847e+00, -5.9022e+00,  2.8437e+00,
         1.2528e-01,  3.3400e+00,  3.1429e+00, -5.5561e+00, -5.0103e+00,
        -2.1802e+00,  3.8787e+00, -2.0884e+00,  2.0822e+00,  2.5675e+00,
         3.6705e+00,  2.0497e+00,  2.6851e+00,  4.4502e+00,  6.0536e+00,
        -6.3581e-01,  5.9316e+00, -1.4351e+00,  1.8774e+00, -2.8698e+00,
         1.5794e+00,  5.9941e-01,  4.3786e+00, -3.6581e+00, -5.3739e+00,
         2.4643e+00, -1.4979e+00, -8.5702e-01,  5.4739e+00, -2.2144e+00,
         3.8705e+00, -1.1978e+00,  4.4758e+00, -2.1

In [64]:
torch.set_printoptions(threshold=3000)
print(res_middle)
print(res_want)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [69]:
fake_features = torch.rand(1, 2048, 10, 8, 8).to(device)
res_middle = last_block_middle(fake_features).squeeze()
res_want = last_block_want(fake_features).squeeze()

In [70]:
for t in range(2048):
    print(res_middle[t], ' ', res_want[t])

tensor(0.4874, device='cuda:0')   tensor(0.4852, device='cuda:0')
tensor(0.4864, device='cuda:0')   tensor(0.4934, device='cuda:0')
tensor(0.5054, device='cuda:0')   tensor(0.5047, device='cuda:0')
tensor(0.5212, device='cuda:0')   tensor(0.5199, device='cuda:0')
tensor(0.4930, device='cuda:0')   tensor(0.5080, device='cuda:0')
tensor(0.5235, device='cuda:0')   tensor(0.5126, device='cuda:0')
tensor(0.5065, device='cuda:0')   tensor(0.5013, device='cuda:0')
tensor(0.5074, device='cuda:0')   tensor(0.5136, device='cuda:0')
tensor(0.5202, device='cuda:0')   tensor(0.5182, device='cuda:0')
tensor(0.5009, device='cuda:0')   tensor(0.5030, device='cuda:0')
tensor(0.4812, device='cuda:0')   tensor(0.4984, device='cuda:0')
tensor(0.4878, device='cuda:0')   tensor(0.4918, device='cuda:0')
tensor(0.5039, device='cuda:0')   tensor(0.5007, device='cuda:0')
tensor(0.5184, device='cuda:0')   tensor(0.5110, device='cuda:0')
tensor(0.5010, device='cuda:0')   tensor(0.5118, device='cuda:0')
tensor(0.5

tensor(0.4952, device='cuda:0')   tensor(0.5003, device='cuda:0')
tensor(0.5073, device='cuda:0')   tensor(0.5147, device='cuda:0')
tensor(0.5229, device='cuda:0')   tensor(0.5249, device='cuda:0')
tensor(0.4822, device='cuda:0')   tensor(0.4896, device='cuda:0')
tensor(0.5094, device='cuda:0')   tensor(0.5059, device='cuda:0')
tensor(0.5188, device='cuda:0')   tensor(0.5110, device='cuda:0')
tensor(0.5080, device='cuda:0')   tensor(0.5039, device='cuda:0')
tensor(0.5249, device='cuda:0')   tensor(0.5279, device='cuda:0')
tensor(0.4948, device='cuda:0')   tensor(0.4971, device='cuda:0')
tensor(0.5036, device='cuda:0')   tensor(0.5010, device='cuda:0')
tensor(0.5152, device='cuda:0')   tensor(0.5082, device='cuda:0')
tensor(0.4993, device='cuda:0')   tensor(0.4931, device='cuda:0')
tensor(0.4966, device='cuda:0')   tensor(0.5026, device='cuda:0')
tensor(0.4975, device='cuda:0')   tensor(0.4969, device='cuda:0')
tensor(0.5128, device='cuda:0')   tensor(0.5117, device='cuda:0')
tensor(0.5

tensor(0.5010, device='cuda:0')   tensor(0.4977, device='cuda:0')
tensor(0.5026, device='cuda:0')   tensor(0.4963, device='cuda:0')
tensor(0.4976, device='cuda:0')   tensor(0.4959, device='cuda:0')
tensor(0.4988, device='cuda:0')   tensor(0.4949, device='cuda:0')
tensor(0.4935, device='cuda:0')   tensor(0.4921, device='cuda:0')
tensor(0.5072, device='cuda:0')   tensor(0.5023, device='cuda:0')
tensor(0.4976, device='cuda:0')   tensor(0.4951, device='cuda:0')
tensor(0.4855, device='cuda:0')   tensor(0.4911, device='cuda:0')
tensor(0.5027, device='cuda:0')   tensor(0.5071, device='cuda:0')
tensor(0.5066, device='cuda:0')   tensor(0.5101, device='cuda:0')
tensor(0.4861, device='cuda:0')   tensor(0.4826, device='cuda:0')
tensor(0.5105, device='cuda:0')   tensor(0.5020, device='cuda:0')
tensor(0.4939, device='cuda:0')   tensor(0.4944, device='cuda:0')
tensor(0.4939, device='cuda:0')   tensor(0.4932, device='cuda:0')
tensor(0.4966, device='cuda:0')   tensor(0.4989, device='cuda:0')
tensor(0.5

tensor(0.4918, device='cuda:0')   tensor(0.4953, device='cuda:0')
tensor(0.5056, device='cuda:0')   tensor(0.4979, device='cuda:0')
tensor(0.5214, device='cuda:0')   tensor(0.5235, device='cuda:0')
tensor(0.5111, device='cuda:0')   tensor(0.5150, device='cuda:0')
tensor(0.4999, device='cuda:0')   tensor(0.4964, device='cuda:0')
tensor(0.5211, device='cuda:0')   tensor(0.5202, device='cuda:0')
tensor(0.5127, device='cuda:0')   tensor(0.4977, device='cuda:0')
tensor(0.4954, device='cuda:0')   tensor(0.4864, device='cuda:0')
tensor(0.5096, device='cuda:0')   tensor(0.5134, device='cuda:0')
tensor(0.4988, device='cuda:0')   tensor(0.4995, device='cuda:0')
tensor(0.4869, device='cuda:0')   tensor(0.4895, device='cuda:0')
tensor(0.5129, device='cuda:0')   tensor(0.5022, device='cuda:0')
tensor(0.4851, device='cuda:0')   tensor(0.4850, device='cuda:0')
tensor(0.4990, device='cuda:0')   tensor(0.4964, device='cuda:0')
tensor(0.5087, device='cuda:0')   tensor(0.5076, device='cuda:0')
tensor(0.5

tensor(0.5305, device='cuda:0')   tensor(0.5278, device='cuda:0')
tensor(0.5132, device='cuda:0')   tensor(0.5169, device='cuda:0')
tensor(0.4930, device='cuda:0')   tensor(0.4837, device='cuda:0')
tensor(0.5003, device='cuda:0')   tensor(0.4958, device='cuda:0')
tensor(0.4849, device='cuda:0')   tensor(0.4951, device='cuda:0')
tensor(0.5103, device='cuda:0')   tensor(0.5127, device='cuda:0')
tensor(0.5060, device='cuda:0')   tensor(0.4976, device='cuda:0')
tensor(0.5370, device='cuda:0')   tensor(0.5229, device='cuda:0')
tensor(0.4954, device='cuda:0')   tensor(0.5028, device='cuda:0')
tensor(0.5117, device='cuda:0')   tensor(0.5137, device='cuda:0')
tensor(0.5147, device='cuda:0')   tensor(0.5094, device='cuda:0')
tensor(0.5023, device='cuda:0')   tensor(0.5111, device='cuda:0')
tensor(0.5155, device='cuda:0')   tensor(0.5120, device='cuda:0')
tensor(0.5052, device='cuda:0')   tensor(0.5020, device='cuda:0')
tensor(0.4917, device='cuda:0')   tensor(0.4948, device='cuda:0')
tensor(0.5

tensor(0.4837, device='cuda:0')   tensor(0.4995, device='cuda:0')
tensor(0.4891, device='cuda:0')   tensor(0.4864, device='cuda:0')
tensor(0.5162, device='cuda:0')   tensor(0.5231, device='cuda:0')
tensor(0.5096, device='cuda:0')   tensor(0.5028, device='cuda:0')
tensor(0.4968, device='cuda:0')   tensor(0.4888, device='cuda:0')
tensor(0.5237, device='cuda:0')   tensor(0.5194, device='cuda:0')
tensor(0.4870, device='cuda:0')   tensor(0.4942, device='cuda:0')
tensor(0.4926, device='cuda:0')   tensor(0.4918, device='cuda:0')
tensor(0.4977, device='cuda:0')   tensor(0.4943, device='cuda:0')
tensor(0.5104, device='cuda:0')   tensor(0.5149, device='cuda:0')
tensor(0.4962, device='cuda:0')   tensor(0.5057, device='cuda:0')
tensor(0.5075, device='cuda:0')   tensor(0.4997, device='cuda:0')
tensor(0.5005, device='cuda:0')   tensor(0.4972, device='cuda:0')
tensor(0.4734, device='cuda:0')   tensor(0.4783, device='cuda:0')
tensor(0.5170, device='cuda:0')   tensor(0.5069, device='cuda:0')
tensor(0.4

tensor(0.4904, device='cuda:0')   tensor(0.4904, device='cuda:0')
tensor(0.5046, device='cuda:0')   tensor(0.4957, device='cuda:0')
tensor(0.4995, device='cuda:0')   tensor(0.4936, device='cuda:0')
tensor(0.4847, device='cuda:0')   tensor(0.4844, device='cuda:0')
tensor(0.4914, device='cuda:0')   tensor(0.4971, device='cuda:0')
tensor(0.5008, device='cuda:0')   tensor(0.4946, device='cuda:0')
tensor(0.4909, device='cuda:0')   tensor(0.4813, device='cuda:0')
tensor(0.5053, device='cuda:0')   tensor(0.5002, device='cuda:0')
tensor(0.5064, device='cuda:0')   tensor(0.4986, device='cuda:0')
tensor(0.4991, device='cuda:0')   tensor(0.4942, device='cuda:0')
tensor(0.5049, device='cuda:0')   tensor(0.5052, device='cuda:0')
tensor(0.5159, device='cuda:0')   tensor(0.5018, device='cuda:0')
tensor(0.5164, device='cuda:0')   tensor(0.5151, device='cuda:0')
tensor(0.5168, device='cuda:0')   tensor(0.5136, device='cuda:0')
tensor(0.5054, device='cuda:0')   tensor(0.5017, device='cuda:0')
tensor(0.5

tensor(0.4866, device='cuda:0')   tensor(0.4820, device='cuda:0')
tensor(0.4752, device='cuda:0')   tensor(0.4759, device='cuda:0')
tensor(0.5077, device='cuda:0')   tensor(0.5171, device='cuda:0')
tensor(0.5037, device='cuda:0')   tensor(0.5055, device='cuda:0')
tensor(0.4849, device='cuda:0')   tensor(0.4917, device='cuda:0')
tensor(0.5009, device='cuda:0')   tensor(0.4927, device='cuda:0')
tensor(0.4925, device='cuda:0')   tensor(0.4936, device='cuda:0')
tensor(0.4946, device='cuda:0')   tensor(0.4973, device='cuda:0')
tensor(0.4896, device='cuda:0')   tensor(0.4943, device='cuda:0')
tensor(0.4874, device='cuda:0')   tensor(0.4860, device='cuda:0')
tensor(0.4868, device='cuda:0')   tensor(0.4967, device='cuda:0')
tensor(0.4890, device='cuda:0')   tensor(0.4805, device='cuda:0')
tensor(0.5055, device='cuda:0')   tensor(0.4970, device='cuda:0')
tensor(0.4874, device='cuda:0')   tensor(0.4862, device='cuda:0')
tensor(0.5062, device='cuda:0')   tensor(0.4939, device='cuda:0')
tensor(0.4

tensor(0.4935, device='cuda:0')   tensor(0.4902, device='cuda:0')
tensor(0.4988, device='cuda:0')   tensor(0.5055, device='cuda:0')
tensor(0.4917, device='cuda:0')   tensor(0.5046, device='cuda:0')
tensor(0.5091, device='cuda:0')   tensor(0.5118, device='cuda:0')


In [71]:
fake_features = torch.rand(1, 2048, 10, 8, 8).to(device) * 1000
res_middle = last_block_middle(fake_features).squeeze()
res_want = last_block_want(fake_features).squeeze()

In [72]:
for t in range(2048):
    print(res_middle[t], ' ', res_want[t])

tensor(497.1497, device='cuda:0')   tensor(484.2032, device='cuda:0')
tensor(510.3890, device='cuda:0')   tensor(503.8865, device='cuda:0')
tensor(517.0241, device='cuda:0')   tensor(521.9487, device='cuda:0')
tensor(510.1040, device='cuda:0')   tensor(507.1018, device='cuda:0')
tensor(484.1008, device='cuda:0')   tensor(488.4927, device='cuda:0')
tensor(493.8463, device='cuda:0')   tensor(500.4989, device='cuda:0')
tensor(519.2897, device='cuda:0')   tensor(524.5828, device='cuda:0')
tensor(470.4863, device='cuda:0')   tensor(484.3643, device='cuda:0')
tensor(510.8155, device='cuda:0')   tensor(504.1078, device='cuda:0')
tensor(496.1772, device='cuda:0')   tensor(495.2704, device='cuda:0')
tensor(495.1665, device='cuda:0')   tensor(493.1105, device='cuda:0')
tensor(491.9718, device='cuda:0')   tensor(495.7995, device='cuda:0')
tensor(504.5929, device='cuda:0')   tensor(505.4805, device='cuda:0')
tensor(509.1648, device='cuda:0')   tensor(501.8317, device='cuda:0')
tensor(500.2262, dev

tensor(490.4644, device='cuda:0')   tensor(484.6920, device='cuda:0')
tensor(497.9304, device='cuda:0')   tensor(495.5759, device='cuda:0')
tensor(507.9355, device='cuda:0')   tensor(495.7515, device='cuda:0')
tensor(492.1257, device='cuda:0')   tensor(495.5264, device='cuda:0')
tensor(493.1303, device='cuda:0')   tensor(504.6736, device='cuda:0')
tensor(518.9981, device='cuda:0')   tensor(513.8754, device='cuda:0')
tensor(513.0229, device='cuda:0')   tensor(515.9241, device='cuda:0')
tensor(514.4834, device='cuda:0')   tensor(513.0291, device='cuda:0')
tensor(507.7722, device='cuda:0')   tensor(510.6344, device='cuda:0')
tensor(500.2498, device='cuda:0')   tensor(490.7180, device='cuda:0')
tensor(469.9868, device='cuda:0')   tensor(479.1645, device='cuda:0')
tensor(480.9585, device='cuda:0')   tensor(479.7567, device='cuda:0')
tensor(491.0017, device='cuda:0')   tensor(482.3185, device='cuda:0')
tensor(482.6135, device='cuda:0')   tensor(489.1909, device='cuda:0')
tensor(507.8797, dev

tensor(502.1382, device='cuda:0')   tensor(508.6834, device='cuda:0')
tensor(496.7329, device='cuda:0')   tensor(505.6831, device='cuda:0')
tensor(498.8130, device='cuda:0')   tensor(505.7602, device='cuda:0')
tensor(504.2895, device='cuda:0')   tensor(516.2424, device='cuda:0')
tensor(509.1250, device='cuda:0')   tensor(515.3951, device='cuda:0')
tensor(521.3308, device='cuda:0')   tensor(505.4330, device='cuda:0')
tensor(524.2805, device='cuda:0')   tensor(528.2290, device='cuda:0')
tensor(509.9837, device='cuda:0')   tensor(497.6031, device='cuda:0')
tensor(493.4076, device='cuda:0')   tensor(487.3135, device='cuda:0')
tensor(494.2820, device='cuda:0')   tensor(496.6220, device='cuda:0')
tensor(503.1147, device='cuda:0')   tensor(505.8639, device='cuda:0')
tensor(486.7893, device='cuda:0')   tensor(496.4892, device='cuda:0')
tensor(509.5065, device='cuda:0')   tensor(507.8539, device='cuda:0')
tensor(509.4011, device='cuda:0')   tensor(501.6510, device='cuda:0')
tensor(518.7047, dev

tensor(480.8333, device='cuda:0')   tensor(496.1642, device='cuda:0')
tensor(501.4644, device='cuda:0')   tensor(500.3532, device='cuda:0')
tensor(524.6006, device='cuda:0')   tensor(530.3290, device='cuda:0')
tensor(501.8310, device='cuda:0')   tensor(495.9302, device='cuda:0')
tensor(525.7746, device='cuda:0')   tensor(515.5847, device='cuda:0')
tensor(490.6907, device='cuda:0')   tensor(497.8383, device='cuda:0')
tensor(509.8009, device='cuda:0')   tensor(504.7699, device='cuda:0')
tensor(510.2252, device='cuda:0')   tensor(510.2503, device='cuda:0')
tensor(491.8969, device='cuda:0')   tensor(490.0387, device='cuda:0')
tensor(492.3746, device='cuda:0')   tensor(502.6977, device='cuda:0')
tensor(512.7040, device='cuda:0')   tensor(512.9703, device='cuda:0')
tensor(496.6967, device='cuda:0')   tensor(498.1552, device='cuda:0')
tensor(501.2265, device='cuda:0')   tensor(500.6335, device='cuda:0')
tensor(485.0113, device='cuda:0')   tensor(482.9836, device='cuda:0')
tensor(510.4991, dev

tensor(501.6453, device='cuda:0')   tensor(505.1735, device='cuda:0')
tensor(504.0521, device='cuda:0')   tensor(496.5905, device='cuda:0')
tensor(490.9429, device='cuda:0')   tensor(492.7978, device='cuda:0')
tensor(497.4709, device='cuda:0')   tensor(496.0182, device='cuda:0')
tensor(496.2800, device='cuda:0')   tensor(504.6638, device='cuda:0')
tensor(492.9398, device='cuda:0')   tensor(497.7477, device='cuda:0')
tensor(511.2477, device='cuda:0')   tensor(503.2096, device='cuda:0')
tensor(507.1973, device='cuda:0')   tensor(516.2767, device='cuda:0')
tensor(520.7855, device='cuda:0')   tensor(517.7787, device='cuda:0')
tensor(515.2665, device='cuda:0')   tensor(505.0427, device='cuda:0')
tensor(488.1053, device='cuda:0')   tensor(495.5004, device='cuda:0')
tensor(481.8970, device='cuda:0')   tensor(494.4447, device='cuda:0')
tensor(489.2441, device='cuda:0')   tensor(491.2850, device='cuda:0')
tensor(517.4136, device='cuda:0')   tensor(526.1319, device='cuda:0')
tensor(495.6774, dev

tensor(497.8214, device='cuda:0')   tensor(498.5226, device='cuda:0')
tensor(492.8170, device='cuda:0')   tensor(489.8922, device='cuda:0')
tensor(497.4977, device='cuda:0')   tensor(487.0418, device='cuda:0')
tensor(519.9528, device='cuda:0')   tensor(518.5316, device='cuda:0')
tensor(501.4909, device='cuda:0')   tensor(498.6284, device='cuda:0')
tensor(485.2681, device='cuda:0')   tensor(488.1430, device='cuda:0')
tensor(515.3891, device='cuda:0')   tensor(505.5595, device='cuda:0')
tensor(487.5233, device='cuda:0')   tensor(492.8231, device='cuda:0')
tensor(519.1235, device='cuda:0')   tensor(522.3204, device='cuda:0')
tensor(499.1211, device='cuda:0')   tensor(513.2881, device='cuda:0')
tensor(500.4846, device='cuda:0')   tensor(503.3699, device='cuda:0')
tensor(482.0796, device='cuda:0')   tensor(480.9603, device='cuda:0')
tensor(499.4013, device='cuda:0')   tensor(504.8121, device='cuda:0')
tensor(499.7641, device='cuda:0')   tensor(501.5448, device='cuda:0')
tensor(478.0779, dev

tensor(503.4110, device='cuda:0')   tensor(498.5873, device='cuda:0')
tensor(504.4756, device='cuda:0')   tensor(494.6674, device='cuda:0')
tensor(491.1419, device='cuda:0')   tensor(488.8070, device='cuda:0')
tensor(524.5970, device='cuda:0')   tensor(512.0667, device='cuda:0')
tensor(499.5578, device='cuda:0')   tensor(505.2293, device='cuda:0')
tensor(506.2369, device='cuda:0')   tensor(517.1752, device='cuda:0')
tensor(488.7015, device='cuda:0')   tensor(483.8290, device='cuda:0')
tensor(513.0643, device='cuda:0')   tensor(507.1654, device='cuda:0')
tensor(524.3956, device='cuda:0')   tensor(516.2289, device='cuda:0')
tensor(507.6931, device='cuda:0')   tensor(502.4678, device='cuda:0')
tensor(491.1996, device='cuda:0')   tensor(491.9309, device='cuda:0')
tensor(507.6112, device='cuda:0')   tensor(495.2704, device='cuda:0')
tensor(496.4007, device='cuda:0')   tensor(502.4010, device='cuda:0')
tensor(497.8218, device='cuda:0')   tensor(492.3244, device='cuda:0')
tensor(485.4251, dev

tensor(469.9815, device='cuda:0')   tensor(469.3198, device='cuda:0')
tensor(500.1871, device='cuda:0')   tensor(498.7088, device='cuda:0')
tensor(493.4035, device='cuda:0')   tensor(484.5701, device='cuda:0')
tensor(499.7526, device='cuda:0')   tensor(495.3331, device='cuda:0')
tensor(486.9672, device='cuda:0')   tensor(478.7051, device='cuda:0')
tensor(506.8911, device='cuda:0')   tensor(501.3421, device='cuda:0')
tensor(487.5745, device='cuda:0')   tensor(496.2754, device='cuda:0')
tensor(492.1411, device='cuda:0')   tensor(489.0731, device='cuda:0')
tensor(467.8548, device='cuda:0')   tensor(479.0492, device='cuda:0')
tensor(502.5751, device='cuda:0')   tensor(516.0186, device='cuda:0')
tensor(499.8947, device='cuda:0')   tensor(500.0583, device='cuda:0')
tensor(521.0148, device='cuda:0')   tensor(522.9932, device='cuda:0')
tensor(496.6667, device='cuda:0')   tensor(500.1462, device='cuda:0')
tensor(501.4963, device='cuda:0')   tensor(504.0737, device='cuda:0')
tensor(491.8714, dev